In [2]:
import numpy as np
from copy import deepcopy
import time

np.set_printoptions(suppress=True)


class SVD1(object):
    def __init__(self, A):
        self.A = A  # general matrix
        self.m = np.shape(A)[0]
        self.n = np.shape(A)[1]

        self.B = None  # matrix after bidiagonalization
        self.B_cut = None  # matrix after bidiagonalization and zero rows/columns discard
        self.U_bi = None  # u_bi * A * v_bi = B in bidiagonalization
        self.V_bi = None

        self.eigval = None  # eigenvalue of b_cut T b_cut
        self.eigvec = None  # eigenvector of b_cut T b_cut

        self.B_s = None  # singular value of B
        self.B_U = None  # U of B
        self.B_V = None  # V of B

    def bidiagonalization(self):  # fit in B, U_bi, V_bi
        if self.m < self.n: 
            A = self.A.T 
            m_smaller_than_n = True 
        else:
            A = self.A
            m_smaller_than_n = False
        
        m = max(self.m, self.n)  # max(m,n)
        n = min(self.m, self.n)  # min(m,n)
        U_bi = np.eye(m)  # multiply before A
        V_bi = np.eye(n)  # multiply after A

        for k in range(n):
            if m - 1 > k:  # delete lower corner
                a = np.reshape(np.array(A[k:, k]), (m - k, 1))  # m-k column vector for Householder transform
                e = np.zeros(m - k)
                e[0] = 1
                e = np.reshape(e, (m - k, 1))  # m-k column vector

                if a[0] < 0:  # calculate vector di (avoid different sign)
                    d_k = a - np.linalg.norm(a) * e
                else:
                    d_k = a + np.linalg.norm(a) * e

                H_v_k = np.eye(m - k) - 2 * (d_k @ d_k.T) / (d_k.T @ d_k)  # Hdk matrix
                if k == 0:
                    u_k = H_v_k  # matrix u1 = Hd1
                else:
                    left_part = np.vstack((np.eye(k), np.zeros((m - k, k))))  # top left and lower left
                    right_part = np.vstack((np.zeros((k, m - k)), H_v_k))  # top right and lower right
                    u_k = np.hstack((left_part, right_part))
                A = u_k @ A
                U_bi = u_k @ U_bi

            if n - 1 > k:  # delete upper corner
                b = np.reshape(np.array(A[k, k + 1:]), (n - k - 1, 1))  # n-k-1 column vector for Householder transform
                e = np.zeros(n - k - 1)
                e[0] = 1
                e = np.reshape(e, (n - k - 1, 1))  # n-k-1 column vector

                if b[0] < 0:  # calculate vector di (avoid different sign)
                    d_k = b - np.linalg.norm(b) * e
                else:
                    d_k = b + np.linalg.norm(b) * e
                H_v_k = np.eye(n - k - 1) - 2 * (d_k @ d_k.T) / (d_k.T @ d_k)
                left_part = np.vstack((np.eye(k + 1), np.zeros((n - k - 1, k + 1))))  # top left and lower left
                right_part = np.vstack((np.zeros((k + 1, n - k - 1)), H_v_k))  # top right and lower right
                v_k = np.hstack((left_part, right_part))
                A = A @ v_k
                V_bi = V_bi @ v_k

        if m_smaller_than_n:
            self.B = A.T
            self.U_bi = V_bi.T
            self.V_bi = U_bi.T
        else:
            self.B = A
            self.U_bi = U_bi
            self.V_bi = V_bi

    def diagonalize_BtB_qr(self):
        if self.m >= self.n:
            self.B_cut = self.B[:self.n, :]
        elif self.m < self.n:
            self.B_cut = self.B[:, :self.m]  # discard zero rows or zero columns
        BtB = self.B_cut.T @ self.B_cut
        N = BtB.shape[0]  # 方阵的维度

        def compute_sigma(m):
            eigval_sub_matrix, eigvec_sub_matrix = np.linalg.eig(m[-2:, -2:])  # lower right most
            if abs(eigval_sub_matrix[0] - m[-1, -1]) > abs(eigval_sub_matrix[1] - m[-1, -1]):
                shift = eigval_sub_matrix[1]
            else:
                shift = eigval_sub_matrix[0]
            return shift

        self.eigval = []
        Qs = []
        for k in range(N - 2):
            n = BtB.shape[0]
            tol = 1
            Q_hat = np.eye(n)
            while tol > 1e-14:
                sigma = compute_sigma(BtB)  # compute sigma_k according to BtB_k-1
                Q, R = np.linalg.qr(BtB - sigma * np.eye(n))
                BtB = R @ Q + sigma * np.eye(n)  # B new
                Q_hat = Q_hat @ Q
                tol = np.linalg.norm(BtB[-1, :-1])  # the last row converges to 0 (except for the last element)

            self.eigval.append(BtB[-1, -1])  # get eigenvalue
            Qs.append(Q_hat)
            BtB = BtB[:-1, :-1]  # continue with smaller matrix
        eigval, eigvec = np.linalg.eig(BtB)
        self.eigval.append(eigval[1])
        self.eigval.append(eigval[0])
        self.eigval.reverse()
        Qs.append(eigvec)  # record lambda and Q

        for i in range(len(Qs)):
            if i != 0:
                Qs[i] = np.hstack(
                    (np.vstack((Qs[i], np.zeros((i, N - i)))), np.vstack((np.zeros((N - i, i)), np.eye(i)))))

        self.eigvec = np.eye(N)
        for i in range(len(Qs)):
            self.eigvec = self.eigvec @ Qs[i]

    def compute_B_svd(self):
        n1 = min(self.m, self.n)
        m1 = max(self.m, self.n)
        self.B_s = np.array(self.eigval) ** 0.5  # get sigma

        self.B_V = deepcopy(self.eigvec)  # dim B_V now is n1*n1
        if self.n > self.m:  # when n is larger, recover B_V to m1*m1 (n*n)
            new_1 = np.zeros((m1 - n1, n1))
            new_2 = np.zeros((m1, m1 - n1))
            self.B_V = np.hstack([np.vstack([self.B_V, new_1]), new_2])
            for i in range(m1 - n1):
                self.B_V[i + n1][i + n1] = 1

        v_eigvec_list = []  # get eigenvectors of V as a list
        for i in range(n1):
            v_eigvec_list.append(np.array([list(self.B_V.T[i])]).T)
        for i in range(n1):  # get columns of U
            u_eigvec = 1 / self.B_s[i] * (self.B @ v_eigvec_list[i])
            if i == 0:
                self.B_U = u_eigvec
            else:
                self.B_U = np.hstack([self.B_U, u_eigvec])

        if self.n < self.m:
            new_2 = np.zeros((m1, m1 - n1))
            self.B_U = np.hstack([self.B_U, new_2])
            for i in range(m1 - n1):
                self.B_U[i + n1][i + n1] = 1

    def compute_A_svd(self):
        self.bidiagonalization()
        self.diagonalize_BtB_qr()
        self.compute_B_svd()
        U_A = self.U_bi.T @ self.B_U
        V_A = self.B_V.T @ self.V_bi.T

        # reorder
        non_ranked_list = deepcopy(list(self.B_s))
        ranked_list = deepcopy(non_ranked_list)
        ranked_list.sort(reverse=True)
        if ranked_list != non_ranked_list:
            for i in range(len(self.B_s)):
                for j in range(i):
                    if self.B_s[i] > self.B_s[j]:
                        self.B_s[i], self.B_s[j] = self.B_s[j], self.B_s[i]
                        U_A[:, i], U_A[:, j] = deepcopy(U_A[:, j]), deepcopy(U_A[:, i])
                        V_A[i, :], V_A[j, :] = deepcopy(V_A[j, :]), deepcopy(V_A[i, :])

        return U_A, self.B_s, V_A


def svd1(m):
    new = SVD1(m)
    u, s, v = new.compute_A_svd()
    return u, s, v


objective = np.array([[10, 2, 3, 5, 6],
                      [2, 1, 3, 4, 7],
                      [2, 2, 5, 3, 8],
                      [3, 4, 5, 2, 9]])
o2 = np.array([[1, 2, 3],
               [4, 5, 6],
               [7, 8, 9],
               [10, 11, 8],
               [13, 142, 1]])
o3 = np.array([[10, 2, 3, 5, 6, 11, 1],
               [2, 1, 3, 4, 7, 12, 111],
               [2, 2, 5, 3, 8, 13, 21],
               [3, 4, 5, 2, 9, 14, 33]])
o1 = np.array([[1, 2, 3],
               [4, 11, 6],
               [7, 8, 111]])

o4 = np.array([[1, 2, 3],
               [4, 11, 6],
               [7, 8, 111],
               [9, 6, 5],
               [43, 57, 89],
               [13, 31, 113]])

a = time.time()
# u, s, v = np.linalg.svd(o2.T)
u, s, v = svd1(o2.T)
b = time.time()
U, S, V = np.linalg.svd(o2.T)
c = time.time()
print(u)
print(U)
print(s)
print(S)
print(v)
print(V)

print(abs(u) - abs(U))
print(abs(v) - abs(V))
print(b - a)
print(c - b)
'''
print(abs(u)-abs(V.T))
print(abs(v)-abs(U.T))
'''


[[ 0.10006487  0.63588685  0.76526788]
 [ 0.99482906 -0.07737813 -0.06578572]
 [ 0.01738273  0.76789357 -0.64034156]]
[[-0.10006487 -0.63588685 -0.76526788]
 [-0.99482906  0.07737813  0.06578572]
 [-0.01738273 -0.76789357  0.64034156]]
[143.48691843  17.5636487    2.64999689]
[143.48691843  17.5636487    2.64999689]
[[ 0.01492729  0.03818258  0.06143787  0.08420858  0.99370698]
 [ 0.1585554   0.38511463  0.61167387  0.66335064 -0.11121109]
 [-0.48578481 -0.4188331  -0.35188139  0.68162475 -0.01261562]
 [-0.31844713 -0.59287966  0.70364415 -0.22794437  0.00337695]
 [-0.79827433  0.56859595  0.05607905 -0.19053586  0.00282275]]
[[-0.01492729 -0.03818258 -0.06143787 -0.08420858 -0.99370698]
 [-0.1585554  -0.38511463 -0.61167387 -0.66335064  0.11121109]
 [ 0.48578481  0.4188331   0.35188139 -0.68162475  0.01261562]
 [-0.31844713 -0.59287966  0.70364415 -0.22794437  0.00337695]
 [-0.79827433  0.56859595  0.05607905 -0.19053586  0.00282275]]
[[ 0.  0.  0.]
 [ 0. -0. -0.]
 [ 0. -0.  0.]]
[[-0

'\nprint(abs(u)-abs(V.T))\nprint(abs(v)-abs(U.T))\n'

In [27]:
class SVD2(object):
    def __init__(self, A):
        self.A = A  # general matrix
        self.m = np.shape(A)[0]
        self.n = np.shape(A)[1]

        self.B = None  # matrix after bidiagonalization
        self.B_cut = None  # matrix after bidiagonalization and zero rows/columns discard
        self.Q = None # Q matrix from QR iteration
        self.R = None # R matrix from QR iteration
        self.L = None # L matrix from cholesky decomposition
        self.X = None # X matix in QR iteration
        
        self.U_bi = None  # u_bi * A * v_bi = B in bidiagonalization
        self.V_bi = None

        self.eigval = None  # eigenvalue of b_cut T b_cut
        self.eigvec = None  # eigenvector of b_cut T b_cut

        self.B_s = None  # singular value of B
        self.B_U = None  # U of B
        self.B_V = None  # V of B

    def bidiagonalization(self):  # fit in B, U_bi, V_bi
        if self.m < self.n:
            A = self.A.T
            m_smaller_than_n = True
        else:
            A = self.A
            m_smaller_than_n = False
        m = max(self.m, self.n)  # max(m,n)
        n = min(self.m, self.n)  # min(m,n)
        U_bi = np.eye(m)  # multiply before A
        V_bi = np.eye(n)  # multiply after A

        for k in range(n):
            if m - 1 > k:  # delete lower corner
                a = np.reshape(np.array(A[k:, k]), (m - k, 1))  # m-k column vector for Householder transform
                e = np.zeros(m - k)
                e[0] = 1
                e = np.reshape(e, (m - k, 1))  # m-k column vector

                if a[0] < 0:  # calculate vector di (avoid different sign)
                    d_k = a - np.linalg.norm(a) * e
                else:
                    d_k = a + np.linalg.norm(a) * e

                H_v_k = np.eye(m - k) - 2 * (d_k @ d_k.T) / (d_k.T @ d_k)  # Hdk matrix
                if k == 0:
                    u_k = H_v_k  # matrix u1 = Hd1
                else:
                    left_part = np.vstack((np.eye(k), np.zeros((m - k, k))))  # top left and lower left
                    right_part = np.vstack((np.zeros((k, m - k)), H_v_k))  # top right and lower right
                    u_k = np.hstack((left_part, right_part))
                A = u_k @ A
                U_bi = u_k @ U_bi

            if n - 1 > k:  # delete upper corner
                b = np.reshape(np.array(A[k, k + 1:]), (n - k - 1, 1))  # n-k-1 column vector for Householder transform
                e = np.zeros(n - k - 1)
                e[0] = 1
                e = np.reshape(e, (n - k - 1, 1))  # n-k-1 column vector

                if b[0] < 0:  # calculate vector di (avoid different sign)
                    d_k = b - np.linalg.norm(b) * e
                else:
                    d_k = b + np.linalg.norm(b) * e
                H_v_k = np.eye(n - k - 1) - 2 * (d_k @ d_k.T) / (d_k.T @ d_k)
                left_part = np.vstack((np.eye(k + 1), np.zeros((n - k - 1, k + 1))))  # top left and lower left
                right_part = np.vstack((np.zeros((k + 1, n - k - 1)), H_v_k))  # top right and lower right
                v_k = np.hstack((left_part, right_part))
                A = A @ v_k
                V_bi = V_bi @ v_k

        if m_smaller_than_n:
            self.B = A.T
            self.U_bi = V_bi.T
            self.V_bi = U_bi.T
        else:
            self.B = A
            self.U_bi = U_bi
            self.V_bi = V_bi

    def cholesky(self):
        t=self.R @ self.R.T
        if t.shape[0] != t.shape[1]:
            print("Error: Input is not a square matrix")
            return
        if (t.T != t).all():
            print("Error: Input is not symmetric")
            return
        try:
            np.linalg.cholesky(t)
        except np.linalg.LinAlgError:
            print("Error: Input is not positive definite")
            return

        n = t.shape[0]
        R = np.zeros((n, n))
        for j in range(n):
            s1 = 0
            if j > 0:
                for k in range(j):
                    s1 += R[k, j] ** 2
            R[j, j] = np.sqrt(t[j, j] - s1)
            for i in range(j, n):
                s2 = 0
                for c in range(j):
                    s2 += R[c, i] * R[c, j]
                R[j, i] = (t[i, j] - s2) / R[j, j]
        return R.T

    def compute_b_svd(self):
        m = self.B.shape[0]
        n = self.B.shape[1]

        if m >= n:
            B = self.B[:n, :n]
            N = n
            M = m
        elif m < n:
            B = self.B[:m, :m]   
            N = m
            M = n                  # discard zero rows or zero columns


        sigma = []
        Qs = []
        self.X = B.T @ B
        for i in range(N - 2):
            Q_hat = np.eye(self.X.shape[0])
            tol = 1
            while tol > 1e-14:
                self.Q, self.R = np.linalg.qr(self.X.T)
                self.L = self.cholesky()
                self.X = self.L.T
                Q_hat = Q_hat @ self.Q 
                tol = self.X[0, 1]

            sigma.append(self.X[-1, -1])
            Qs.append(Q_hat)
            self.X = self.X[:-1, :-1]

        eigenval, eigenvect = np.linalg.eig(self.X)
        sigma.append(eigenval[1])
        sigma.append(eigenval[0])
        sigma.reverse()
        self.B_s=np.array(sigma)**0.5

        Qs.append(eigenvect)  # record lambda and Q

        for i in range(len(Qs)):
            if i != 0:
                Qs[i] = np.hstack((np.vstack((Qs[i], np.zeros((i, N - i)))), np.vstack((np.zeros((N - i, i)), np.eye(i)))))

        Q_prod = np.eye(N)
        for i in range(len(Qs)):
            Q_prod = Q_prod @ Qs[i]
        self.B_V=Q_prod


        if n == M:  
            new_1 = np.zeros((M - N, N))
            new_2 = np.zeros((M, M - N))
            self.B_V = np.vstack([self.B_V, new_1])
            self.B_V = np.hstack([self.B_V, new_2])
            for i in range(M - N):
                self.B_V[i + N][i + N] = 1
        v = []

        for i in range(m):
            v.append(np.array([list(self.B_V.T[i])]).T)
        for i in range(m):
            new = 1 / self.B_s[i] * (self.B @ v[i])
            if i == 0:
                self.B_U = new
            else:
                self.B_U = np.hstack([self.B_U, new])

    def compute_A_svd(self):
        self.bidiagonalization()
        self.compute_b_svd()
        U_A = self.U_bi.T @ self.B_U
        V_A = self.B_V.T @ self.V_bi.T
        return U_A, self.B_s, V_A
    
    
    
    


def svd2(m):
    new = SVD2(m)
    u, s, v = new.compute_A_svd()
    return u, s, v

a = time.time()
# u, s, v = np.linalg.svd(o2.T)
u, s, v = svd2(objective)
b = time.time()
U, S, V = np.linalg.svd(objective)
c = time.time()
print(u)
print(U)
print(s)
print(S)
print(v)
print(V)

print(abs(u) - abs(U))
print(abs(v) - abs(V))
print(b - a)
print(c - b)

[[ 0.56737579 -0.81928842 -0.05262068 -0.06389255]
 [ 0.40914721  0.19722023  0.74175661  0.49345706]
 [ 0.47439838  0.37928122  0.15571608 -0.77900221]
 [ 0.53444312  0.38212095 -0.65021588  0.38154087]]
[[-0.56737579 -0.81928842 -0.05262068 -0.06389255]
 [-0.40914721  0.19722023  0.74175661  0.49345706]
 [-0.47439838  0.37928122  0.15571608 -0.77900221]
 [-0.53444312  0.38212095 -0.65021588  0.38154087]]
[20.8939754   7.05858083  2.63317606  0.82741322]
[20.8939754   7.05858083  2.63317606  0.82741322]
[[ 0.43286059  0.22161738  0.38163042  0.33337596  0.71185402]
 [-0.83494382  0.11981015  0.27495701 -0.1991159   0.41625214]
 [-0.25896829 -0.62772715 -0.15384129  0.71041186  0.1026737 ]
 [-0.07903335  0.40346347 -0.84433409  0.09721934  0.32957443]
 [ 0.20540126 -0.61620379 -0.20540126 -0.57885811  0.44814821]]
[[-0.43286059 -0.22161738 -0.38163042 -0.33337596 -0.71185402]
 [-0.83494382  0.11981015  0.27495701 -0.1991159   0.41625214]
 [-0.25896829 -0.62772715 -0.15384129  0.7104118